In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('data/task1/dataset.csv', encoding='cp1251')

In [4]:
stops = pd.read_csv('data/task1/stops.csv')

In [3]:
pd.set_option('display.max_colwidth', 200)

In [5]:
df.date = pd.to_datetime(df.date, infer_datetime_format=True)

In [7]:
df.sample(10).T

,2687196,2183010,161821,3917581,3727212,1624374,1382962,463347,946375,1994801
date,2017-11-08 00:26:10,2017-09-10 15:55:10,2017-01-19 17:30:20,2018-03-13 13:40:20,2018-02-19 12:52:10,2017-07-08 00:09:10,2017-06-10 01:33:50,2017-02-23 15:04:40,2017-04-20 12:49:20,2017-08-19 21:07:00
RF.21304.Ток...213MII904A,5,0,4.9,5,5.1,4.9,4.9,4.9,5,4.9
S.C.ВПУСК.ПП.ДАВЛ...214PI226AA,12.7,-0.44,13.46,15.14,14.73,12.09,11.63,8.34,9.21,7.65
S.C.ВПУСК.ПП.ДАВЛ...214PI226AB,13.23,-0.44,13.38,15.45,15.11,12.36,12.38,8.42,9.23,7.79
S.C.ВПУСК.ПП.ТЕМП...214TI232A,251.2,21.8,252.3,256.9,253.7,260.9,254.5,239.5,238.7,208.3
SPEED.CONTROLLER...250MSIC001.PV,40.29,0,0,0,40,0,0,38.04,38.95,0
XX21401.МАСС.РАСХ.ГРАНУЛ...214FIC112.PV,34.4066,3.53648e-17,31.5973,33.4731,36.006,29.077,32.244,33.3126,34.6344,33.4968
ДАВЛ.ВАЛ.ВПЕР.УПР...214PIC232A,0.4685,0.71,0.4416,0.456,0.4272,0.5347,0.4866,0.5082,0.566,0.5201
ДВИГАТЕЛЬ.ГРАНУЛЯТОРА...214MSIC981,444,-0.4,460,450.5,472.4,396.9,427.2,436.5,437.2,407.3
Е.21402А....ГР.ВОДА.ВЫХ...214TIC101A,50,21.9,51.1,50.6,50.9,50,49.9,48,48,51


In [9]:
stops.date = pd.to_datetime(stops.date, infer_datetime_format=True)
print(stops)

                  date       type
0  2017-11-01 11:03:00       stop
1  2017-01-30 08:51:00       stop
2  2017-02-24 17:45:00       stop
3  2017-07-03 00:16:00       stop
4  2017-08-03 11:23:00  vibrosito
5  2017-08-03 15:56:00  vibrosito
6  2017-10-03 17:55:00       stop
7  2017-03-31 01:42:00  vibrosito
8  2017-01-04 07:45:00       stop
9  2017-03-04 20:48:00       stop
10 2017-04-21 10:02:00       stop
11 2017-04-24 13:51:00       stop
12 2017-04-06 13:23:00       stop
13 2017-06-06 13:32:00  vibrosito
14 2017-07-06 16:29:00  vibrosito
15 2017-06-10 03:21:00       stop
16 2017-08-10 16:00:00       stop
17 2017-10-14 14:13:00       stop
18 2017-10-15 09:34:00  vibrosito
19 2017-10-21 02:23:00       stop
20 2017-07-11 04:13:00       stop
21 2017-09-11 04:00:00       stop
22 2017-10-11 13:06:00       stop
23 2017-11-14 05:20:00       stop
24 2017-11-16 21:25:00       stop
25 2017-11-19 01:55:00  vibrosito
26 2017-11-19 11:21:00  vibrosito
27 2017-11-21 22:53:00       stop
28 2017-09-12 

In [10]:
df.isna().sum()

date                                            0
RF.21304.Ток...213MII904A                   77760
S.C.ВПУСК.ПП.ДАВЛ...214PI226AA             151199
S.C.ВПУСК.ПП.ДАВЛ...214PI226AB             151199
S.C.ВПУСК.ПП.ТЕМП...214TI232A              151199
SPEED.CONTROLLER...250MSIC001.PV           155519
XX21401.МАСС.РАСХ.ГРАНУЛ...214FIC112.PV     77760
ДАВЛ.ВАЛ.ВПЕР.УПР...214PIC232A             151199
ДВИГАТЕЛЬ.ГРАНУЛЯТОРА...214MSIC981          77760
Е.21402А....ГР.ВОДА.ВЫХ...214TIC101A       151199
ЕХ21401.АЕ01.MFR...214AI200A                77760
НАГНЕТАНИЕ.В.EX.21401...250PI001            77760
ПОЛОЖ.НОЖА..ГРАНУЛЯТОРА...214ZI211A        151199
ПОЛОЖ.ЩЕЛ.ДИСКА...214ZI202A                155519
РАСХОД.В....EX.21401...250FIC001.PV         77760
РАСХОД.ЛИНИИ.P.W...214FI205A               151199
СЕКТ.ПИТАТЕЛЬ.RF21304...213MSIC904.PV       77760
СПЕЦ.ЭНЕРГИЯ...214JYY200A                   77760
ТЕМП.ЛИН.Г.В..ДР.ЛИН....214TI247A           77760
ТЕМП.ЛИН.Г.В..ЛИН.РАЗ....214TI246A         151199


In [15]:
df = df.merge(stops, left_on='date', right_on='date', how='outer').fillna(0)

In [25]:
df[df.iloc[:,2] == 0]

,date,RF.21304.Ток...213MII904A,S.C.ВПУСК.ПП.ДАВЛ...214PI226AA,S.C.ВПУСК.ПП.ДАВЛ...214PI226AB,S.C.ВПУСК.ПП.ТЕМП...214TI232A,SPEED.CONTROLLER...250MSIC001.PV,XX21401.МАСС.РАСХ.ГРАНУЛ...214FIC112.PV,ДАВЛ.ВАЛ.ВПЕР.УПР...214PIC232A,ДВИГАТЕЛЬ.ГРАНУЛЯТОРА...214MSIC981,Е.21402А....ГР.ВОДА.ВЫХ...214TIC101A,...,ЦИЛ..4.ТЕМП.УПР...214TIC208A,ЦИЛ..5.ТЕМП.УПР...214TIC209A,ЦИЛ..6.ТЕМП.УПР...214TIC210A,ЦИЛ..7.ТЕМП.УПР...214TIC211A,ЦИЛ..8.ТЕМП.УПР...214TIC212A,ЦИЛ..9.ТЕМП.УПР...214TIC213A,ЭКСТР.ДВИГ.ВЛАСТЬ...214JI200A,ЭКСТР.ДВИГ.ПОТОК...214II200A,H2.C3..КОНТР.СООТН...212AIC008.PV,type
90386,2017-01-11 11:04:30,0.0,0.0,0.64,238.199997,0.0,2.002209e+01,0.5614,-0.400000,51.000000,...,242.100006,230.800003,220.000000,174.199997,173.699997,179.699997,0.0,0.100000,4.302217,0
90390,2017-01-11 11:05:10,0.0,0.0,0.64,235.800003,0.0,1.652300e-10,0.5614,-0.400000,50.799999,...,240.600006,230.500000,220.100006,174.199997,172.699997,178.100006,0.0,-0.200000,4.302217,0
90831,2017-01-11 12:18:40,0.0,0.0,0.57,225.699997,0.0,1.575750e-16,0.7114,-0.300000,49.400002,...,257.899994,259.899994,248.699997,289.200012,278.300018,296.800018,0.0,-0.100000,4.285271,0
269018,2017-02-01 03:16:30,0.0,0.0,-0.43,149.800003,0.0,6.579180e-17,0.7100,-0.300000,37.299999,...,66.300003,71.000000,74.500000,79.400002,79.500000,83.300003,0.0,-0.200000,7.073433,0
281332,2017-02-02 13:28:50,0.0,0.0,-0.44,144.000000,0.0,6.579180e-17,0.7200,-0.200000,30.800001,...,227.199997,229.400009,227.300003,226.699997,254.300003,251.300003,0.0,-0.200000,3.609717,0
282940,2017-02-02 17:56:50,0.0,0.0,0.55,224.600006,0.0,6.579180e-17,0.5000,447.200012,51.799999,...,251.100006,250.800003,253.900009,258.100006,249.300003,234.800003,0.0,-0.200000,3.606387,0
283125,2017-02-02 18:27:40,0.0,0.0,0.60,226.100006,0.0,6.579180e-17,0.5000,448.100006,54.700001,...,251.800003,248.699997,246.900009,236.699997,241.500000,265.100006,0.0,-0.200000,3.965829,0
283126,2017-02-02 18:27:50,0.0,0.0,0.61,226.100006,0.0,6.579180e-17,0.5100,448.100006,54.700001,...,251.800003,248.800003,246.900009,236.800003,241.500000,265.200012,0.0,-0.200000,3.965829,0
283265,2017-02-02 18:51:00,0.0,0.0,0.62,225.900009,0.0,6.579180e-17,0.5600,-0.300000,54.299999,...,233.100006,252.199997,250.500000,262.300018,257.200012,247.800003,0.0,-0.200000,4.615153,0
283267,2017-02-02 18:51:20,0.0,0.0,0.60,225.800003,0.0,6.579180e-17,0.5600,-0.300000,54.299999,...,232.900009,252.199997,250.600006,262.500000,257.300018,247.100006,0.0,-0.200000,4.069296,0


In [22]:
df['type'].replace(['vibrosito', 'stop'], 1, inplace=True)

In [ ]:
df['Not_working'] = (df[])